In [318]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from config import BaseConfig
config = BaseConfig()

In [319]:
data_df = pd.read_csv(config.data_csv_path)

In [320]:
data_df

,current_province,current_city,current_county,current_members_live_with,gender,year_born,edu_level,hometown_code,hometown,year_current_flow,...,hometown_Name_County,hometown_lon,hometown_lat,first_Name_Province,first_Name_Prefecture,first_Name_County,first_lon,first_lat,current_lon,current_lat
0,广东省,深圳市,光明新区,4,2,1989,4,360681,江西省鹰潭市贵溪市,2015,...,贵溪市,117.186973,28.188428,江西省,南昌市,青山湖区,115.905297,28.719082,NaN,NaN
1,广东省,深圳市,光明新区,3,2,1981,5,360402,江西省九江市濂溪区,2015,...,濂溪区,116.039436,29.634605,广东省,深圳市,南山区,113.937903,22.554902,NaN,NaN
2,福建省,厦门市,同安区,4,2,1986,4,511623,四川省广安市邻水县,2013,...,邻水县,106.991830,30.258922,福建省,泉州市,丰泽区,118.617882,24.922059,118.102758,24.776209
3,新疆生产建设兵团,第六师,军户农场,3,1,1973,3,652301,新疆维吾尔自治区昌吉回族自治州昌吉市,2000,...,昌吉市,87.059347,44.091644,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,北京市,北京市,朝阳区,2,2,1984,5,511702,四川省达州市通川区,2010,...,通川区,107.432388,31.362205,四川省,成都市,武侯区,104.022906,30.610118,116.508837,39.951928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169984,上海市,上海市,闵行区,3,2,1959,2,330602,浙江省绍兴市越城区,1982,...,越城区,120.639159,30.004908,上海市,上海市,闵行区,121.414389,31.089025,121.414389,31.089025
169985,安徽省,宣城市,宁国市,4,1,1976,3,341823,安徽省宣城市泾县,2005,...,泾县,118.362169,30.595836,上海市,上海市,闵行区,121.414389,31.089025,118.985855,30.497997
169986,北京市,北京市,海淀区,4,2,1977,6,131022,河北省廊坊市固安县,1997,...,固安县,116.278512,39.345288,北京市,北京市,朝阳区,116.508837,39.951928,116.227491,40.025917
169987,浙江省,杭州市,江干区,4,2,1971,3,330213,浙江省宁波市奉化区,1992,...,奉化区,121.368450,29.613198,浙江省,杭州市,江干区,120.291011,30.312594,120.291011,30.312594


In [321]:
data_df['current_combined'] = data_df['current_province'] + data_df['current_city'] + data_df['current_county']

In [322]:
from utils.loading import get_json

level = get_json(config.level_path)

In [323]:
def fill_current(df: pd.DataFrame, level: list) -> pd.DataFrame:
    # Initialize the columns
    df['current_province_code'] = None
    df['current_city_code'] = None
    df['current_county_code'] = None
    
    # Iterate over the rows in the DataFrame
    for idx, row in df.iterrows():
        row_province = row['current_province']
        row_city = row['current_city']
        row_county = row['current_county']
        
        # Iterate over the `level` list (assuming it's a list of dicts)
        for province in level:
            if row_province != province['name']:
                continue

            df.at[idx, 'current_province_code'] = province['province']
            # case 1: city is province, 重庆 data is malformed and is treated as a normal province
            if row_city == row_province:
                df.at[idx, 'current_city_code'] = province['children'][0]['city']
                for county in province['children']:
                    if county['name'] == row_county:
                        df.at[idx, 'current_county_code'] = county['area']
                        break
            else:
                for city in province['children']:
                    if row_city != city['name']:
                        continue

                    df.at[idx, 'current_city_code'] = city['city']
                    for county in city['children']:
                        if row_county != county['name']:
                            continue

                        df.at[idx, 'current_county_code'] = county['area']
                        break

    return df


In [324]:
# Military bases will be None
filled_df = fill_current(data_df.copy(), level=level)

In [325]:
unfound_chongqing_rows = (filled_df['current_city'] == '重庆市') & (filled_df['current_county_code'].isna())

# chongqing has missing counties from the dataset
filled_df[unfound_chongqing_rows]['current_combined'].nunique()

5

In [326]:
unfound_cities_rows = (filled_df['current_city_code'].isna())


filled_df[unfound_cities_rows]['current_combined'].nunique()

127

In [327]:
found_citites_but_unfound_neighborhood_rows = (~filled_df['current_city_code'].isna()) & (filled_df['current_county_code'].isna())

filled_df[found_citites_but_unfound_neighborhood_rows]['current_combined'].nunique()

153

In [328]:
data_df['hometown_lon'].isna().sum() / len(data_df), data_df['hometown_lat'].isna().sum() / len(data_df)


(np.float64(0.10353611116013389), np.float64(0.10353611116013389))

In [329]:
data_df['first_lon'].isna().sum() / len(data_df), data_df['first_lat'].isna().sum() / len(data_df)

(np.float64(0.19997764561236314), np.float64(0.19997764561236314))

In [330]:
data_df['current_lon'].isna().sum() / len(data_df), data_df['current_lat'].isna().sum() / len(data_df)

(np.float64(0.13364982440040238), np.float64(0.13364982440040238))

In [331]:
hometown_lon_lat_missing = filled_df['hometown_lon'].isna() | filled_df['hometown_lat'].isna()
first_lon_lat_missing = filled_df['first_lon'].isna() | filled_df['first_lat'].isna()
current_lon_lat_missing = filled_df['current_lon'].isna() | filled_df['current_lat'].isna()

In [332]:
filled_df[hometown_lon_lat_missing]['hometown'].nunique()

745

In [333]:
filled_df[first_lon_lat_missing]['first_flow_location'].nunique()

696

In [334]:
filled_df[current_lon_lat_missing]['current_combined'].nunique()

271

In [335]:
# list of all locations with missing ids
locations_missing_ids = np.unique(np.concatenate([
    filled_df[unfound_chongqing_rows]['current_combined'].unique(), 
    filled_df[unfound_cities_rows]['current_combined'].unique(),
    filled_df[found_citites_but_unfound_neighborhood_rows]['current_combined'].unique()
]))

locations_missing_coordiantes = np.unique(np.concatenate([
    filled_df[hometown_lon_lat_missing]['hometown'].unique(),
    filled_df[first_lon_lat_missing]['first_flow_location'].unique(),
    filled_df[current_lon_lat_missing]['current_combined'].unique()
]))

locations_to_be_queried = np.unique(np.concatenate([locations_missing_ids, locations_missing_coordiantes]))

len(locations_missing_ids), len(locations_missing_coordiantes), len(locations_to_be_queried)

(280, 1018, 1030)

In [336]:
total_mask = (unfound_chongqing_rows) | (unfound_cities_rows) | (found_citites_but_unfound_neighborhood_rows) | (hometown_lon_lat_missing) | (first_lon_lat_missing) | (current_lon_lat_missing)
total_geo_mask = (hometown_lon_lat_missing) | (first_lon_lat_missing) | (current_lon_lat_missing)
total_ids_mask = (unfound_chongqing_rows) | (unfound_cities_rows) | (found_citites_but_unfound_neighborhood_rows)

len(filled_df[total_mask]) / len(filled_df), len(filled_df[total_geo_mask]) / len(filled_df), len(filled_df[total_ids_mask])/ len(filled_df)

(0.3441399149356723, 0.34084558412603166, 0.1364735365229515)

In [341]:
cleaned_df = filled_df[~total_mask]

In [359]:
for column in cleaned_df.columns:
    if cleaned_df[column].isna().sum() != 0:
        print(column, cleaned_df[column].isna().sum())

average_family_cost_per_month 6
average_family_income_per_month 4
how_long_to_stay 18856


In [388]:
def process_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['hometown_province_code'] = df['hometown_code'].apply(lambda x: str(x)[:2])
    df['hometown_city_code'] = df['hometown_code'].apply(lambda x: str(x)[2:4])
    df['hometown_county_code'] = df['hometown_code'].apply(lambda x: str(x)[4:6])
    df['first_flow_province_code'] = df['first_flow_code'].apply(lambda x: str(x)[:2])
    df['first_flow_city_code'] = df['first_flow_code'].apply(lambda x: str(x)[2:4])
    df['first_flow_county_code'] = df['first_flow_code'].apply(lambda x: str(x)[4:6])
    df['current_code'] = df['current_province_code'].astype(str) + df['current_city_code'].astype(str) + df['current_county_code'].astype(str)

    columns_to_keep = [
        'hometown_code',
        'hometown_province_code',
        'hometown_city_code',
        'hometown_county_code',
        'hometown_lon',
        'hometown_lat',

        'first_flow_code',
        'first_flow_province_code',
        'first_flow_city_code',
        'first_flow_county_code',
        'first_lon',
        'first_lat',
        'year_first_flow',
        'month_first_flow',

        'current_code',
        'current_province_code',
        'current_city_code',
        'current_county_code',
        'current_lon',
        'current_lat',
        'year_current_flow',
        'month_current_flow',

        'gender',
        'edu_level',
        'average_family_cost_per_month',
        'average_family_income_per_month',
        'num_flows_total',
        'if_stay',
        'if_change_household_local',
        'how_long_to_stay',
        
    ]
    # drop insignificant rows of missing data
    df = df.dropna(subset=['average_family_cost_per_month', 'average_family_income_per_month'])
    df = df[columns_to_keep]
    return df

In [389]:
cleaned_df = process_df(cleaned_df)

In [391]:
cleaned_df.to_csv(config.data_folder / 'cleaned_data.csv', index=False)